<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-12 14:30:20
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  3.49 K
Current:  1.39 C
-------------------
Today PnL: -1.40 L (-1.0%)
Current PnL: -30.09 L (-19.19%)
CY Booked + Current PnL: -15.35 L (-9.79%)
-------------------
Total profit:  1.56 L
Total loss:  -31.66 L
-------------------
Total Booked + Current PnL: 11.63 L (8.95%)
Total Booked PnL: 41.72 L (32.14%)
Curr Year Booked PnL: 14.74 L (10.57%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 95.47 L (68.49%)
Deployed:  1.30 C
Current:  1.39 C
CAGR/XIRR %: 3.68%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,0.91,1.90,19.30,21.57,19269.0,1860.0,99840.0,397.04,-11.92,59.0,H-SC,3.52,168.0,0.10,0.72,45.70,XR,ATH,FINANCE
18,COALINDIA,-1.05,2.99,15.77,19.23,24437.0,4496.0,154956.0,484.83,21.35,46.0,L-LC,12.07,182.0,0.18,1.11,24.90,XY25,ATH,MINING
77,TTKPRESTIG,-1.79,-24.89,33.26,0.09,25175.0,-25085.0,75692.0,770.00,75.67,46.0,M-SC,8.55,253.0,-1.00,0.54,4.17,OX40N,NTT,DURABLES
51,MEDANTA,0.08,-4.22,29.54,24.08,35919.0,-5357.0,121593.0,1486.00,-11.91,53.0,X-SC,11.63,91.0,-0.15,0.87,11.37,XY24,NTT,HEALTHCARE
5,ANGELONE,-1.29,12.42,10.48,24.20,35962.0,37900.0,343150.0,3033.00,59.94,60.0,X-SC,3.58,99.0,1.05,2.46,43.72,X40N,NTT,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,VAIBHAVGBL,4.30,-20.30,101.70,60.77,148158.0,-37094.0,145681.0,521.00,68.04,62.0,H-SC,2.13,158.0,-0.25,1.05,32.08,XR,NTT,JEWELLERY
7,ATULAUTO,2.63,-11.43,64.36,45.57,112366.0,-22542.0,174590.0,844.00,3786.49,71.0,M-SC,6.51,245.0,-0.20,1.25,32.12,XY24,NTT,AUTO
82,VOLTAS,2.26,20.52,20.47,45.20,47304.0,39348.0,231090.0,1856.00,8.73,68.0,X-MC,3.10,80.0,0.83,1.66,28.71,XY25,NTT,AC
85,ZYDUSLIFE,1.24,-5.27,43.18,35.63,85723.0,-11050.0,198524.0,1286.17,-18.51,50.0,H-MC,6.17,119.0,-0.13,1.42,10.52,AR,ATH,PHARMA
32,HAVELLS,1.20,-10.34,47.30,32.08,139102.0,-33907.0,294084.0,2062.85,-14.91,59.0,X-MC,9.82,69.0,-0.24,2.11,9.93,X40,ATH,ELECTRICAL


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BSOFT,-6.29,-20.99,90.84,50.79,100950.0,-29524.0,111129.0,831.70,1.28,51.0,H-SC,7.86,171.0,-0.29,0.80,29.39,XR,ATH,IT
64,SHALBY,-5.95,-35.39,93.62,25.10,109574.0,-64102.0,117041.0,327.00,761.24,40.0,M-SC,15.96,248.0,-0.59,0.84,2.51,XY24,NTT,HEALTHCARE
42,INFY,-5.56,-5.37,41.86,34.24,123944.0,-16797.0,296091.0,1972.00,-26.42,21.0,X-LC,5.07,6.0,-0.14,2.12,2.47,X40,NTT,IT
72,TATAELXSI,-5.38,-28.40,84.40,32.03,79667.0,-37443.0,94392.0,9161.00,-22.59,35.0,H-SC,6.15,157.0,-0.47,0.68,5.73,OX40N,NTT,IT
73,TCS,-5.36,-24.28,56.56,18.55,161991.0,-91831.0,286406.0,4311.59,-34.63,27.0,X-LC,1.97,3.0,-0.57,2.06,0.00,X40,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,0.74,4.44,99.07,107.91,152766.0,6556.0,154200.0,1641.55,-14.88,68.0,M-SC,10.26,220.0,0.04,1.11,16.01,OX40N,ATH,CABLES
66,SIS,-1.56,-21.79,55.04,21.26,47797.0,-24192.0,86840.0,528.00,2051.81,58.0,H-SC,4.08,163.0,-0.51,0.62,19.72,OX40N,NTT,MISC
77,TTKPRESTIG,-1.79,-24.89,33.26,0.09,25175.0,-25085.0,75692.0,770.00,75.67,46.0,M-SC,8.55,253.0,-1.00,0.54,4.17,OX40N,NTT,DURABLES
72,TATAELXSI,-5.38,-28.40,84.40,32.03,79667.0,-37443.0,94392.0,9161.00,-22.59,35.0,H-SC,6.15,157.0,-0.47,0.68,5.73,OX40N,NTT,IT
70,SYMPHONY,-1.18,-28.58,39.98,-0.03,48865.0,-48918.0,122223.0,1306.00,-42.66,57.0,M-SC,10.61,206.0,-1.00,0.88,12.95,OX40N,NTT,DURABLES


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,2.26,20.52,20.47,45.2,47304.0,39348.0,231090.0,1856.0,8.73,68.0,X-MC,3.1,80.0,0.83,1.66,28.71,XY25,NTT,AC


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-1.05,2.99,15.77,19.23,24437.0,4496.0,154956.0,484.83,21.35,46.0,L-LC,12.07,182.0,0.18,1.11,24.90,XY25,ATH,MINING
50,MASFIN,0.91,1.90,19.30,21.57,19269.0,1860.0,99840.0,397.04,-11.92,59.0,H-SC,3.52,168.0,0.10,0.72,45.70,XR,ATH,FINANCE
26,FINCABLES,0.74,4.44,99.07,107.91,152766.0,6556.0,154200.0,1641.55,-14.88,68.0,M-SC,10.26,220.0,0.04,1.11,16.01,OX40N,ATH,CABLES
1,ABB,-0.90,5.92,37.43,45.57,103720.0,15485.0,277104.0,7934.00,-31.89,68.0,H-MC,3.21,121.0,0.15,1.99,23.05,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,-0.90,5.92,37.43,45.57,103720.0,15485.0,277104.0,7934.00,-31.89,68.0,H-MC,3.21,121.0,0.15,1.99,23.05,AR,NTT,ELECTRICAL
50,MASFIN,0.91,1.90,19.30,21.57,19269.0,1860.0,99840.0,397.04,-11.92,59.0,H-SC,3.52,168.0,0.10,0.72,45.70,XR,ATH,FINANCE
26,FINCABLES,0.74,4.44,99.07,107.91,152766.0,6556.0,154200.0,1641.55,-14.88,68.0,M-SC,10.26,220.0,0.04,1.11,16.01,OX40N,ATH,CABLES
39,INDIAMART,-1.92,-5.46,118.66,106.72,138358.0,-6736.0,116600.0,4810.62,-53.11,47.0,H-SC,7.55,138.0,-0.05,0.84,16.64,AR,ATH,MISC
84,WIPRO,-4.22,-9.72,90.81,72.27,138918.0,-16472.0,152976.0,420.00,-22.41,29.0,M-LC,4.92,101.0,-0.12,1.10,0.00,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,-5.56,-5.37,41.86,34.24,123944.0,-16797.0,296091.0,1972.00,-26.42,21.0,X-LC,5.07,6.0,-0.14,2.12,2.47,X40,NTT,IT
33,HCLTECH,-4.05,-3.38,28.17,23.83,65845.0,-8182.0,233742.0,1908.19,0.89,25.0,X-LC,5.99,13.0,-0.12,1.68,12.14,X40,ATH,IT
73,TCS,-5.36,-24.28,56.56,18.55,161991.0,-91831.0,286406.0,4311.59,-34.63,27.0,X-LC,1.97,3.0,-0.57,2.06,0.00,X40,ATH,IT
8,AWL,-1.31,-32.96,134.47,57.19,294005.0,-107487.0,218640.0,485.00,-68.35,30.0,X-MC,17.08,58.0,-0.37,1.57,0.00,XY24,NTT,FMCG
13,BERGEPAINT,0.87,-17.61,47.03,21.14,88093.0,-40027.0,187312.0,680.00,-23.43,30.0,X-MC,10.45,75.0,-0.45,1.34,0.87,XY24,NTT,PAINTS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,-5.36,-24.28,56.56,18.55,161991.0,-91831.0,286406.0,4311.59,-34.63,27.0,X-LC,1.97,3.0,-0.57,2.06,0.00,X40,ATH,IT
81,VBL,-0.19,-7.88,47.27,35.67,137537.0,-24882.0,290960.0,671.64,-18.89,42.0,X-LC,2.00,5.0,-0.18,2.09,4.88,X40N,ATH,FMCG
21,DABUR,-0.51,1.61,41.43,43.70,104426.0,3987.0,252055.0,735.00,-6.68,57.0,X-MC,2.02,73.0,0.04,1.81,16.90,XY24,BTT,FMCG
34,HINDUNILVR,-2.01,-6.11,21.07,13.67,50852.0,-15713.0,241350.0,2922.00,-13.11,54.0,X-LC,2.56,9.0,-0.31,1.73,13.94,XY25,NTT,FMCG
65,SIEMENS,-0.20,-15.49,48.52,25.51,76307.0,-28825.0,157270.0,4671.50,-3.51,54.0,H-LC,2.89,50.0,-0.38,1.13,15.90,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
2,ABBOTINDIA,-1.23,-10.82,32.39,18.07,51665.0,-19345.0,159510.0,35195.00,-23.70,37.0,X-MC,9.95,64.0,-0.37,1.14,0.00,X40,ATH,PHARMA
8,AWL,-1.31,-32.96,134.47,57.19,294005.0,-107487.0,218640.0,485.00,-68.35,30.0,X-MC,17.08,58.0,-0.37,1.57,0.00,XY24,NTT,FMCG
73,TCS,-5.36,-24.28,56.56,18.55,161991.0,-91831.0,286406.0,4311.59,-34.63,27.0,X-LC,1.97,3.0,-0.57,2.06,0.00,X40,ATH,IT
13,BERGEPAINT,0.87,-17.61,47.03,21.14,88093.0,-40027.0,187312.0,680.00,-23.43,30.0,X-MC,10.45,75.0,-0.45,1.34,0.87,XY24,NTT,PAINTS
53,PGHH,-1.47,-12.36,50.64,32.01,94758.0,-26400.0,187120.0,17616.87,-38.41,37.0,X-MC,9.07,60.0,-0.28,1.34,1.14,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,-1.97,-30.39,105.21,42.84,47608.0,-19756.0,45250.0,424.00,-54.27,50.0,X-SC,19.97,83.0,-0.41,0.32,8.84,XY24,NTT,MISC
58,RELAXO,-1.24,-52.48,220.52,52.31,152108.0,-76183.0,68977.0,1176.00,-49.38,40.0,X-SC,13.44,92.0,-0.50,0.49,2.57,X40N,NTT,FOOTWEAR
12,BATAINDIA,-2.01,-42.80,136.36,35.20,100366.0,-55066.0,73604.0,2096.00,-3.54,48.0,X-SC,20.93,93.0,-0.55,0.53,4.60,X40,NTT,FOOTWEAR
51,MEDANTA,0.08,-4.22,29.54,24.08,35919.0,-5357.0,121593.0,1486.00,-11.91,53.0,X-SC,11.63,91.0,-0.15,0.87,11.37,XY24,NTT,HEALTHCARE
35,HONAUT,-0.81,-21.51,82.59,43.32,105583.0,-35032.0,127840.0,58357.33,-32.50,42.0,X-SC,10.00,90.0,-0.33,0.92,3.80,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,0.42,-21.67,41.43,10.78,77989.0,-52074.0,188244.0,452.00,-53.41,42.0,X-LC,18.96,1.0,-0.67,1.35,3.69,X40,NTT,FMCG
75,TMPV,-0.57,-21.60,56.86,22.98,100698.0,-48800.0,177098.0,600.00,-18.19,66.0,X-LC,3.46,2.0,-0.48,1.27,13.22,XY24,NTT,AUTO
73,TCS,-5.36,-24.28,56.56,18.55,161991.0,-91831.0,286406.0,4311.59,-34.63,27.0,X-LC,1.97,3.0,-0.57,2.06,0.00,X40,ATH,IT
81,VBL,-0.19,-7.88,47.27,35.67,137537.0,-24882.0,290960.0,671.64,-18.89,42.0,X-LC,2.00,5.0,-0.18,2.09,4.88,X40N,ATH,FMCG
42,INFY,-5.56,-5.37,41.86,34.24,123944.0,-16797.0,296091.0,1972.00,-26.42,21.0,X-LC,5.07,6.0,-0.14,2.12,2.47,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,0.28,-31.52,94.05,32.89,49727.0,-24334.0,52873.0,1800.00,-752.71,59.0,L-MC,11.34,259.0,-0.49,0.38,45.61,XR,NTT,BANKS
67,SONACOMS,-0.82,-7.69,50.68,39.09,47325.0,-7779.0,93380.0,804.02,-27.83,66.0,M-SC,3.73,227.0,-0.16,0.67,32.10,AR,ATH,AUTO
50,MASFIN,0.91,1.90,19.30,21.57,19269.0,1860.0,99840.0,397.04,-11.92,59.0,H-SC,3.52,168.0,0.10,0.72,45.70,XR,ATH,FINANCE
14,BSOFT,-6.29,-20.99,90.84,50.79,100950.0,-29524.0,111129.0,831.70,1.28,51.0,H-SC,7.86,171.0,-0.29,0.80,29.39,XR,ATH,IT
79,VAIBHAVGBL,4.30,-20.30,101.70,60.77,148158.0,-37094.0,145681.0,521.00,68.04,62.0,H-SC,2.13,158.0,-0.25,1.05,32.08,XR,NTT,JEWELLERY


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ATULAUTO,2.63,-11.43,64.36,45.57,112366.0,-22542.0,174590.0,844.00,3786.49,71.0,M-SC,6.51,245.0,-0.20,1.25,32.12,XY24,NTT,AUTO
67,SONACOMS,-0.82,-7.69,50.68,39.09,47325.0,-7779.0,93380.0,804.02,-27.83,66.0,M-SC,3.73,227.0,-0.16,0.67,32.10,AR,ATH,AUTO
11,BANDHANBNK,0.11,-18.97,137.47,92.42,310284.0,-52849.0,225710.0,400.00,58.76,73.0,H-SC,8.34,169.0,-0.17,1.62,30.44,XY24,NTT,BANKS
5,ANGELONE,-1.29,12.42,10.48,24.20,35962.0,37900.0,343150.0,3033.00,59.94,60.0,X-SC,3.58,99.0,1.05,2.46,43.72,X40N,NTT,FINANCE
79,VAIBHAVGBL,4.30,-20.30,101.70,60.77,148158.0,-37094.0,145681.0,521.00,68.04,62.0,H-SC,2.13,158.0,-0.25,1.05,32.08,XR,NTT,JEWELLERY


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.24
1,25,44.58
2,50,76.50


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.82
MC    30.96
LC    23.20
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.71
X40      23.02
X40N     15.26
XR        8.93
AR        8.92
XY25      8.77
OX40N     7.48
SR        0.89
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.32
H-SC    22.99
X-LC    19.86
M-SC    11.64
X-SC    10.40
H-MC     4.87
M-MC     1.39
H-LC     1.13
L-LC     1.11
M-LC     1.10
L-SC     0.79
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.01,-13.20,51.24
FINANCE,13.45,-11.37,55.20
IT,11.69,-31.39,97.03
MISC,7.11,-30.91,84.46
ELECTRICAL,6.15,-9.61,49.60
PAINTS,5.06,-30.84,50.28
INSURANCE,4.82,-0.90,35.43
PHARMA,4.10,-2.62,34.93
AUTO,3.19,-17.78,58.51


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3246820.0,21
AR,1328458.0,10
X40,1286727.0,15
XR,1248957.0,12
X40N,992654.0,10
OX40N,748976.0,10
XY25,397928.0,6
SR,296419.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3592146.0,24
X-MC,1634654.0,16
M-SC,1435939.0,15
X-LC,1121791.0,12
X-SC,840329.0,9
H-MC,399753.0,3
L-SC,180742.0,2
M-LC,138918.0,1
H-LC,76307.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1230619.0      6
           AR         941783.0      5
M-SC       XY24       809793.0      6
H-SC       XR         789342.0      7
X-LC       X40        579813.0      6
X-MC       X40        538646.0      7
           XY24       486524.0      3
           X40N       403784.0      4
X-SC       X40N       365441.0      4
H-SC       OX40N      333983.0      4
M-SC       OX40N      327080.0      5
X-SC       XY24       306620.0      3
H-SC       SR         296419.0      2
X-LC       X40N       223429.0      2
H-MC       XY24       210310.0      1
X-MC       XY25       205700.0      2
X-LC       XY24       202954.0      2
H-MC       AR         189443.0      2
M-SC       XR         178141.0      2
X-SC       X40        168268.0      2
M-LC       XR         138918.0      1
M-SC       AR         120925.0      2
X-LC       XY25       115595.0      2
L-SC       XR          92829.0      1
           OX40N       87913.0      1
H-LC       AR          76307.0      1
M-MC       XY25        52196.0      1
L-MC       XR          49727.0      1
L-LC       XY25        24437.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
